In [3]:
import matplotlib
matplotlib.use('module://ipympl.backend_nbagg')

from matplotlib import pyplot as plt
plt.ion()

import math
import numpy as np
import pickle
import sys

sys.path.append('..')
import unzipping_simulation as uzsi
from unzipping_simulation import kB

sys.path.append('../../../cn_plot_style/')
import cn_plot_style as cnps

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

np.set_printoptions(precision=3)

dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_keys(['settings', 'E_ext_ssDNA', 'ext_dsDNA_wlc', 'XFE', 'XFE0'])
dict_k

In [13]:
# compatibililty to load old results
#_E_ext_ssDNA = uzsi.E_ext_ssDNA
#EXT_DSDNA = uzsi.EXT_DSDNA
#_ext_dsDNA_wlc = uzsi.ext_dsDNA_wlc
#unboundfunction = uzsi.unboundfunction

with open('simulations.p','rb') as f:
    simulations = pickle.load(f)
    
#for key in simulations.keys():
#    del simulations[key]['E_ext_ssDNA']
#    del simulations[key]['ext_dsDNA_wlc']

#with open('simulations.p','wb') as f:
#    pickle.dump(simulations, f)

In [16]:
np.set_printoptions(precision=2)

#[(value['settings']['k_rot']*math.pi/180*1e12, value['settings']['angles_r0']*180/math.pi, key) for key, value in simulations.items()]
for simulation in simulations.values():
    # Set the unzipping construct parameters
    bases = simulation['settings']['bases']
    nbs = simulation['settings']['nbs']
    nbp = simulation['settings']['nbp']
    nbs_loop = simulation['settings']['nbs_loop']
    S = simulation['settings']['S']
    L_p_ssDNA = simulation['settings']['L_p_ssDNA']
    z = simulation['settings']['z']
    pitch = simulation['settings']['pitch']
    L_p_dsDNA = simulation['settings']['L_p_dsDNA']

    # Set other experimental parameters
    radius = simulation['settings']['radius']
    angles_r0 = simulation['settings']['angles_r0']
    kappa = simulation['settings']['kappa']
    k_rot = simulation['settings']['k_rot']
    c = simulation['settings']['c']
    T = simulation['settings']['T']

    # Set parameters for the simulation
    NNBP = simulation['settings']['NNBP']
    x0_min = simulation['settings']['x0_min']
    x0_max = simulation['settings']['x0_max']
    h0 = simulation['settings']['h0']
    y0 = simulation['settings']['y0']
    resolution = simulation['settings']['resolution']
    boltzmann_factor = simulation['settings']['boltzmann_factor']

    # set variables of simulated data
    XFE, XFE0 = simulation['XFE'], simulation['XFE0']

    # assign variables to be plotted
    # extension, number of unzipped basepairs, force
    X = XFE['X']
    X0 = XFE['A0'][:,0]
    NUZ0_avg = XFE['NUZ0_avg']
    NUZ0_max = XFE['NUZ0_max_W0']
    EXT_APP_avg = XFE['EXT_APP_avg']
    F0_avg = XFE['F0_avg']
    F0d_avg = np.array([np.sqrt(((xfe0['D0_avg'] * kappa)**2).sum()) for xfe0 in XFE0])  # should be the same as:
    #F0d_avg = np.array([(np.sqrt(((xfe0['D0'] * kappa)**2).sum(axis=1)) * xfe0['W0'] / xfe0['W0'].sum()).sum() for xfe0 in XFE0])
    F0XYZ_avg = np.array([np.sqrt(((xfe0['D0_avg'] * kappa)**2)) for xfe0 in XFE0])
    F0_max = XFE['F0_max_W0']
    THETA_DIFF = XFE['DA0_avg'][:,0]

    #for fignum in plt.get_fignums()[:-5]:
    #    plt.close(fignum)

    # scale the measured data
    shift_x = 0
    scale_x = 1
    shift_F = 0
    
    plt.ion()

    # select data which was properly fitted
    if np.all(k_rot == 0):
        idx_valid = (X != 0)
    else:
        idx_valid = np.logical_and(abs(THETA_DIFF) > 0*math.pi/180, abs(THETA_DIFF) < 45*math.pi/180)
    fig, axes = plt.subplots(2,2)
    
    with cnps.cn_plot(context='presentation', fig_scale=1.0, dark=False, usetex=False) as cnp:
        fig.suptitle('k_rot: {}, angles_r0: {}'.format(k_rot*1e12*math.pi/180, angles_r0*180/math.pi))
        for i in range(len(I)):
            ax = axes[0][i]
            ax2 = cnps.second_ax(subplot_pos=(2,2,i+1), link_ax=ax, spines_ax=ax)

            # Plot measured unzipping curve(s)
            #for _x, _F, _FXYZ in zip(x, F, FXYZ):
        #    ax.plot(x[i] * scale_x + shift_x, F[i] + shift_F)
            #ax.plot(x[i] * scale_x + shift_x, np.abs(FXYZ[i]))

            # Plot simulated unzipping curve
            ax.plot(X[idx_valid]*1e9, F0_avg[idx_valid]*1e12, '.', label='Force construct')
            ax.plot(EXT_APP_avg[idx_valid]*1e9, F0d_avg[idx_valid]*1e12, label='Force bead')
            #ax.plot(EXT_APP_avg[idx_valid]*1e9, (F0_avg[idx_valid] - F0d_avg[idx_valid])*1e12, label='Force difference')
            #ax.plot(X*1e9, F0XYZ_avg*1e12)
            #ax.plot(X*1e9, F0_max*1e12)

            # Plot number of simulated unzipped basepairs
            #ax2.plot(EXT_APP_avg*1e9, NUZ0_avg)
            ax2.plot(EXT_APP_avg[idx_valid]*1e9, THETA_DIFF[idx_valid]*180/math.pi)

            ax.grid(True)

            #ax.set_xlim((float(X.min()*1e9), float(X.max()*1e9)))
            #ax2.set_xlim((float(X0.min()*1e9), float(X0.max()*1e9)))

            ax.set_xlabel('(Apparent) extension of the construct (nm)')
            ax.set_ylabel('Force (pN)')
            #ax2.set_xlabel('Horizontal position of the stage (nm)')
            ax2.set_ylabel('Angle theta difference (°)')
            
            ax.set_ylim((-1, 18))
        for i in range(len(I)):
            ax = axes[1][i]
            ax2 = cnps.second_ax(subplot_pos=(2,2,i+3), link_ax=ax, spines_ax=ax)
            #ax2 = plt.twinx(ax=ax)
            #ax2.xaxis.set_visible(False)

            # Plot measured unzipping curve(s)
            #for _x, _F, _FXYZ in zip(x, F, FXYZ):
            #ax.plot(_x * scale_x + shift_x, _F + shift_F)
      #      ax.plot(x[i] * scale_x + shift_x, np.abs(FXYZ[i]))

            # Plot simulated unzipping curve
            #ax.plot(X*1e9, F0d_avg*1e12, label='Force construct')
            #ax.plot(X*1e9, (F0_avg - F0d_avg)*1e12, label='Force difference')
            ax.plot(EXT_APP_avg[idx_valid]*1e9, F0XYZ_avg[idx_valid]*1e12)
            #ax.plot(X*1e9, F0_max*1e12)

            # Plot number of simulated unzipped basepairs
            ax2.plot(EXT_APP_avg[idx_valid]*1e9, NUZ0_avg[idx_valid])

            ax.grid(True)

            #ax.set_xlim((float(X.min()*1e9), float(X.max()*1e9)))
            #ax2.set_xlim((float(X0.min()*1e9), float(X0.max()*1e9)))

            ax.set_xlabel('Apparent extension of the construct (nm)')
            ax.set_ylabel('Force (pN)')
            #ax2.set_xlabel('Horizontal position of the stage (nm)')
            ax2.set_ylabel('Number of unzipped basepairs')
            
            ax.set_ylim((-1, 18))
        #fig.savefig('results_r_opp/k_rot_{:04.2f}_{:04.2f}_angles_r0_{:03.0f}_{:03.0f}.pdf'.format(k_rot[0]*1e12*math.pi/180, k_rot[1]*1e12*math.pi/180, angles_r0[0]*180/math.pi, angles_r0[1]*180/math.pi))
        #fig.clear()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

...

FigureCanvasNbAgg()

/usr/local/lib/python3.7/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/usr/local/lib/python3.7/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [8]:
#with open('simulations.p','rb') as f:
#    simulation = pickle.load(f)
#key = get_key(x0_min, x0_max, y0=y0, h0=h0, resolution=resolution,
#              bases=bases, nbs=nbs, nbp=nbp, nbs_loop=nbs_loop,
#              radius=radius, angles_r0=angles_r0, kappa=kappa, k_rot=k_rot,
#              S=S, L_p_ssDNA=L_p_ssDNA, z=z,
#              pitch=pitch, L_p_dsDNA=L_p_dsDNA,
#              NNBP=NNBP, c=c, T=T,
#              boltzmann_factor=boltzmann_factor)
#simulation = simulations[key]

with open('simulation.p','rb') as f:
    simulation = pickle.load(f)
    
XFE = simulation['XFE']
XFE0 = simulation['XFE0']

# assign variables to be plotted
# extension, number of unzipped basepairs, force
X = XFE['X']
X0 = XFE['A0'][0]
NUZ0_avg = XFE['NUZ0_avg']
NUZ0_max = XFE['NUZ0_max_W0']
EXT_APP_avg = XFE['EXT_APP_avg']
# Force acting on the DNA
F0_avg = XFE['F0_avg']
F0_max = XFE['F0_max_W0']
# Force acting on the bead
F0d_avg = np.array([np.sqrt(((xfe0['D0_avg'] * kappa)**2).sum()) for xfe0 in XFE0])  # should be the same as:
#F0d_avg = np.array([(np.sqrt(((xfe0['D0'] * kappa)**2).sum(axis=1)) * xfe0['W0'] / xfe0['W0'].sum()).sum() for xfe0 in XFE0])
F0XYZ_avg = np.array([np.sqrt(((xfe0['D0_avg'] * kappa)**2)) for xfe0 in XFE0])
THETA_DIFF = XFE['DA0_avg'][:,0]

for fignum in plt.get_fignums()[:-5]:
    plt.close(fignum)

# scale the measured data
shift_x = 0
scale_x = 1
shift_F = 0

# select data which was properly fitted
idx_valid = np.logical_and(abs(THETA_DIFF) > 0*math.pi/180, abs(THETA_DIFF) < 45*math.pi/180) 

with cnps.cn_plot(context='notebook', fig_scale=1.7, dark=False, usetex=False) as cnp:
    
    fig, axes = plt.subplots(2,2)
    for i in range(len(I)):
        ax = axes[0][i]
        ax2 = cnps.second_ax(subplot_pos=(2,2,i+1), link_ax=ax, spines_ax=ax)

        # Plot measured unzipping curve(s)
        #for _x, _F, _FXYZ in zip(x, F, FXYZ):
        ax.plot(x[i] * scale_x + shift_x, F[i] + shift_F)
        #ax.plot(x[i] * scale_x + shift_x, np.abs(FXYZ[i]))

        # Plot simulated unzipping curve
        ax.plot(X[idx_valid]*1e9, F0_avg[idx_valid]*1e12, '.', label='Force construct')
        ax.plot(EXT_APP_avg[idx_valid]*1e9, F0d_avg[idx_valid]*1e12, label='Force bead')
        #ax.plot(EXT_APP_avg[idx_valid]*1e9, (F0_avg[idx_valid] - F0d_avg[idx_valid])*1e12, label='Force difference')
        #ax.plot(X*1e9, F0XYZ_avg*1e12)
        #ax.plot(X*1e9, F0_max*1e12)

        # Plot number of simulated unzipped basepairs
        #ax2.plot(EXT_APP_avg*1e9, NUZ0_avg)
        ax2.plot(EXT_APP_avg[idx_valid]*1e9, THETA_DIFF[idx_valid]*180/math.pi)

        ax.grid(True)

        #ax.set_xlim((float(X.min()*1e9), float(X.max()*1e9)))
        #ax2.set_xlim((float(X0.min()*1e9), float(X0.max()*1e9)))

        ax.set_xlabel('(Apparent) extension of the construct (nm)')
        ax.set_ylabel('Force (pN)')
        #ax2.set_xlabel('Horizontal position of the stage (nm)')
        ax2.set_ylabel('Angle theta difference (°)')
    for i in range(len(I)):
        ax = axes[1][i]
        ax2 = cnps.second_ax(subplot_pos=(2,2,i+3), link_ax=ax, spines_ax=ax)
        #ax2 = plt.twinx(ax=ax)
        #ax2.xaxis.set_visible(False)

        # Plot measured unzipping curve(s)
        #for _x, _F, _FXYZ in zip(x, F, FXYZ):
        #ax.plot(_x * scale_x + shift_x, _F + shift_F)
        ax.plot(x[i] * scale_x + shift_x, np.abs(FXYZ[i]))

        # Plot simulated unzipping curve
        #ax.plot(X*1e9, F0d_avg*1e12, label='Force construct')
        #ax.plot(X*1e9, (F0_avg - F0d_avg)*1e12, label='Force difference')
        ax.plot(EXT_APP_avg[idx_valid]*1e9, F0XYZ_avg[idx_valid]*1e12)
        #ax.plot(X*1e9, F0_max*1e12)

        # Plot number of simulated unzipped basepairs
        ax2.plot(EXT_APP_avg[idx_valid]*1e9, NUZ0_avg[idx_valid])

        ax.grid(True)

        #ax.set_xlim((float(X.min()*1e9), float(X.max()*1e9)))
        #ax2.set_xlim((float(X0.min()*1e9), float(X0.max()*1e9)))

        ax.set_xlabel('Apparent extension of the construct (nm)')
        ax.set_ylabel('Force (pN)')
        #ax2.set_xlabel('Horizontal position of the stage (nm)')
        ax2.set_ylabel('Number of unzipped basepairs')
#    fig.savefig('result.png')

FigureCanvasNbAgg()

NameError: name 'x' is not defined

In [15]:
import os
#Load investigator package
import pyoti
pyoti.info()

#Create an experiment
experiment = pyoti.create_experiment()
# Choose the path, were the experiment should be created (or opened from)
#
# datadir: The path to where the experiment (and the data) are located
# datafile: The name of the file that contains the data. Here it is only used to generate dbfile.
#           The data is loaded further down upon creation of a Record.
# dbfile: The name of the database file the experiment is saved to (or loaded from).
datadir =  '../../../../data/ASWAD/unzipping/2015-03-12 - unzipping/data/'
#datadir =  '../../../../data/ASWAD/unzipping/2017-08-01_03 - unzipping and biotin concentration/0.1 percent - unzipping and jumping in trap, l_2, l_4 plate/data/'
datafile = 'B09.bin'

# Choose pull_release pairs
I = [0,1]

# For the name of the experiment, exchange the extension '.bin' with '.fs'
dbfile = os.path.join(datadir, datafile.replace('.bin', '.fs'))

# Create/open the experiment dbfile
experiment.open(dbfile)

results_region = experiment.view('results')

# Choose resolution for presentation of data (extension, force)
resolution = 500  # points/s resolution

# Create Result objects to obtain force and extension
tether = pyoti.create_tether(region=results_region, resolution=resolution)

# Show the autodetected minima, maxima and sections
#tether._sf.highest_frequency=32
#tether._sf.reduce_false_positives = True
#tether._sf.compare_time = 0.025
tether.update()
#tether.init_rfig(legend=True)
#tether.init_fe_fig()
#tether.show_force_extension_plots(bps=1399, autolimit=False)
    
kappa_xyz = experiment.view.results.calibration.kappa()
timevector = experiment.view.results.timevector
posXYZ = experiment.view.results.positionXYZ

# Select the unzipping pair number(s)
x = []
F = []
FXYZ = []
DISTXYZ = []
axis = []
for i in I:
    _x, _F, _ax, *ignore = tether.force_extension_pair(i)
    x.append(_x)
    F.append(_F)
    axis.append(_ax[0])

    stress, release = tether.stress_release_pairs(i=i)
    tether.calibration.kappa(posXYZ[:,2].mean())
    # Get extension and force (in a fast way)
    data = tether.get_data(traces=['psdXYZ', 'positionXYZ'], samples=stress[0])

    psdXYZ = data[:, 0:3]
    positionXYZ = data[:, 3:6]
    positionXY = positionXYZ[:, 0:2]
    positionZ = data[:, [5]]

    displacementXYZ \
        = tether.calibration.displacement(psdXYZ, positionZ=positionZ)
    distXYZ = pyoti.evaluate.tether.distanceXYZ(positionXYZ, displacementXYZ,
                          tether.calibration.radius,
                          tether.calibration.focalshift)
    DISTXYZ.append(distXYZ*1e3)

    dist = pyoti.evaluate.tether.distance(distXYZ, positionXY)
    forceXY_Z = tether.calibration.force(displacementXYZ,
                                       positionZ=positionZ)

    fXYZ = pyoti.evaluate.tether.forceXYZ(forceXY_Z)
    FXYZ.append(fXYZ*1e3)

experiment.abort()
experiment.close()

PyOTI - the investigator package of the PyOTIC software
Version: 0.12.13

The actual working path is: '/home/tobias/ZMBP/ipython/notebooks/unzipping_simulation/unzipping_simulation/notebooks'
Create experiment file:
   '/home/tobias/ZMBP/ipython/data/ASWAD/unzipping/2015-03-12 - unzipping/data/B09.fs'
Cannot open the experiment file!
 The file cannot be created!
 The original error message is: '[Errno 2] No such file or directory: '/home/tobias/ZMBP/ipython/data/ASWAD/unzipping/2015-03-12 - unzipping/data/B09.fs.lock''
Experiment is closed!


TypeError: Missing required argument: 'region'

In [6]:
#x[0], FXYZ[0]
i = 1
angles_E = np.array([cart2sph(*point)[1:] for point in DISTXYZ[i]])*180/math.pi
#angles_E[:,1] -= 90
angles_F = np.array([cart2sph(*point)[1:] for point in FXYZ[i]])*180/math.pi
#angles_F[:,1] -= 90
angles_diff = angles_E - angles_F
plt.ion()
fig, ax = plt.subplots()
ax.plot(x[i], angles_E[:,0])
ax.plot(x[i], angles_F[:,0])
ax.plot(x[i], angles_diff[:,0])
ax.set_xlabel('Apparent extension (nm)')
ax.set_ylabel('Theta (°)')
ax.set_title('Theta')
fig, ax = plt.subplots()
ax.plot(x[i], angles_E[:,1])
ax.plot(x[i], angles_F[:,1])
#ax.plot(x[i], angles_diff[:,1])
ax.set_xlabel('Apparent extension (nm)')
ax.set_ylabel('Phi (°)')
ax.set_title('Phi')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5, 1.0, 'Phi')